In [103]:
import os
import pandas as pd
from plotly import graph_objects as go
import numpy as np
from Bio import Phylo
import plotly.io as pio
import shutil
import importlib
from tqdm import tqdm
from plotly.subplots import make_subplots
from pathlib import Path
if not Path('jw_utils').exists():
    !git clone https://github.com/JonWinkelman/jw_utils.git
from jw_utils import plotly_preferences as pprefs
from jw_utils import itol_annotations as ia
from jw_utils import ncbi_utils as nu
from jw_utils import parse_fasta as pfa
from jw_utils import hmmer_utils as hu
from jw_utils import parse_gff as pgf
from jw_utils import alignment_utils2 as au

if not Path('orthofinder_utils').exists():
    !git clone https://github.com/JonWinkelman/orthofinder_utils.git
from orthofinder_utils import dash_ortho_parser as dop
from orthofinder_utils import run_orthofinder as ru
import external_functions as ef

### Download proteomes and GFF files from NCBI datasets

In [98]:
results_dir = Path('./results/1_orthofinder_analysis')
results_dir.mkdir(exist_ok=True)
nu.download_genomes_from_accfile('./data/accessions.txt',files_to_include='protein,gff3', dataset_fp='./datasets.zip')

Error: Collecting 126 genome records [------------------------------------------------]   0% 0/126
Downloading: ./datasets.zip    847B 6.85MB/s
Downloading: ./datasets.zip    847B 6.85MB/s
Downloading: ./datasets.zip    847B 6.85MB/s
Downloading: ./datasets.zip    847B 6.85MB/s
Downloading: ./datasets.zip    847B 6.85MB/s
Downloading: ./datasets.zip    847B 6.85MB/s
Downloading: ./datasets.zip    847B 6.85MB/s
Downloading: ./datasets.zip    847B 6.85MB/s
Downloading: ./datasets.zip    847B 6.85MB/s
Downloading: ./datasets.zip    847B 6.85MB/s
Downloading: ./datasets.zip    9.13kB 84kB/s
Downloading: ./datasets.zip    9.13kB 84kB/s
Downloading: ./datasets.zip    9.13kB 84kB/s
Downloading: ./datasets.zip    32.8kB 233kB/s
Downloading: ./datasets.zip    32.8kB 233kB/s
Downloading: ./datasets.zip    32.8kB 233kB/s
Downloading: ./datasets.zip    89.5kB 515kB/s
Downloading: ./datasets.zip    89.5kB 515kB/s
Downloading: ./datasets.zip    89.5kB 515kB/s
Downloading: ./datasets.zip    89.5kB 51

#### Download assembly summaries

In [115]:
Path('data/summary_data/').mkdir(exist_ok=True)
nu.download_assembly_summaries('./data/accessions.txt', output_file='./data/summary_data/summaries.json')

## Run orthofinder 2.5.5 on proteomes

In [105]:
nu.move_gffs(data_dir='./datasets/ncbi_dataset/data/', new_gff_dir='./data/gffs')
nu.move_proteomes(data_dir='./datasets/ncbi_dataset/data/', new_proteome_dir='./data/proteomes')
ru.run_orthofinder('./data/proteomes/')

Running: orthofinder -f ./data/proteomes/

OrthoFinder version 2.5.5 Copyright (C) 2014 David Emms

2025-09-05 13:46:02 : Starting OrthoFinder 2.5.5
14 thread(s) for highly parallel tasks (BLAST searches etc.)
1 thread(s) for OrthoFinder algorithm

Checking required programs are installed
----------------------------------------
Test can run "mcl -h" - ok
Test can run "fastme -i /Users/jonathanwinkelman/Trestle/Palmer_lab/Geary_et_al_2025/data/proteomes/OrthoFinder/Results_Sep05/WorkingDirectory/dependencies/SimpleTest.phy -o /Users/jonathanwinkelman/Trestle/Palmer_lab/Geary_et_al_2025/data/proteomes/OrthoFinder/Results_Sep05/WorkingDirectory/dependencies/SimpleTest.tre" - ok

Dividing up work for BLAST for parallel processing
--------------------------------------------------
2025-09-05 13:46:10 : Creating diamond database 1 of 126
2025-09-05 13:46:10 : Creating diamond database 2 of 126
2025-09-05 13:46:10 : Creating diamond database 3 of 126
2025-09-05 13:46:10 : Creating diamond da

Traceback (most recent call last):
  File "/Users/jonathanwinkelman/miniconda3/envs/palmer/bin/orthofinder", line 7, in <module>
Process Process-1:
    main(args)
  File "/Users/jonathanwinkelman/miniconda3/envs/palmer/bin/scripts_of/__main__.py", line 1785, in main
Process Process-8:
Process Process-6:
    RunSearch(options, speciesInfoObj, seqsInfo, prog_caller)
  File "/Users/jonathanwinkelman/miniconda3/envs/palmer/bin/scripts_of/__main__.py", line 1529, in RunSearch
    proc.join()
  File "/Users/jonathanwinkelman/miniconda3/envs/palmer/lib/python3.11/multiprocessing/process.py", line 149, in join
    res = self._popen.wait(timeout)
          ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jonathanwinkelman/miniconda3/envs/palmer/lib/python3.11/multiprocessing/popen_fork.py", line 43, in wait
    return self.poll(os.WNOHANG if timeout == 0.0 else 0)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jonathanwinkelman/miniconda3/envs/palmer/lib/python3.11/multiproces

KeyboardInterrupt: 

In [20]:
figure_genome_assemblies = {
    'GCF_001077675.1': 'Acinetobacter baumannii ATCC 17978-mff',
    'GCF_000018445.1': 'Acinetobacter baumannii ACICU',
    'GCF_000021245.2': 'Acinetobacter baumannii AB0057',
    'GCF_000770605.1': 'Acinetobacter baumannii AB5075',
    'GCF_005281455.1': 'Acinetobacter nosocomialis M2',
    'GCF_002055515.1': 'Acinetobacter calcoaceticus CA16',
    'GCF_001682515.1': 'Acinetobacter gyllenbergii FMP01',
    'GCF_000413935.1': 'Acinetobacter colistiniresistens NIPH 2036',
    'GCF_000046845.1': 'Acinetobacter baylyi ADP1',
    'GCF_000368805.1': 'Acinetobacter johnsonii ANC 3681',
    'GCF_004208515.1': 'Acinetobacter halotolerans JCM 31009',
    'GCF_001105265.1': 'Yersinia enterocolitica SC9312-78',
    }

### Parse orthofinder output to find HOGs. 
- The below code will may not be stable upon re-running OrthoFinder, HOGs will likely have different names etc.

In [114]:
#dop_obj = dop.DashOrthoParser('data/')

In [38]:
#N0 hog contianing astA and AstO (a 'duplication of astA1') N0.HOG0000651
#N0 hog contianing astR (the putative regulater of the ast2 operon in A. baumannii)  N0.HOG0002500

In [138]:
# HOG0002500_protein_seqs = dop_obj.get_HOG_protein_seqs('N0.HOG0002500')


In [141]:
# HOG0002500_seqs_d = HOG0002500_protein_seqs['Protein_seq'].to_dict()
# HOG0002500_seqs_fp = results_dir / 'HOG0002500_astR.faa'
# pfa.write_to_fasta(HOG0002500_seqs_d, str(HOG0002500_seqs_fp) )
HOG0002500_seqs_d = pfa.get_seq_dict(str(HOG0002500_seqs_fp))
muscle_out= str(HOG0002500_seqs_fp).replace('.faa', '.muscle.aln')
importlib.reload(ef)
ef.run_muscle(HOG0002500_seqs_d, muscle_out)

Running: muscle -in /var/folders/vw/7lg51dfd3ql9g_j55xz_3dqh0000gn/T/muscle_k0omsz4q.fa -out results/1_orthofinder_analysis/HOG0002500_astR.muscle.aln
MUSCLE v3.8.1551 by Robert C. Edgar

http://www.drive5.com/muscle
This software is donated to the public domain.
Please cite: Edgar, R.C. Nucleic Acids Res 32(5), 1792-97.

muscle_k0omsz4q 76 seqs, lengths min 123, max 156, avg 141
00:00:00      2 MB(0%)  Iter   1    0.03%  K-mer dist pass 1
00:00:00      2 MB(0%)  Iter   1   17.12%  K-mer dist pass 1
00:00:00      2 MB(0%)  Iter   1   34.21%  K-mer dist pass 1
00:00:00      2 MB(0%)  Iter   1   51.30%  K-mer dist pass 1
00:00:00      2 MB(0%)  Iter   1   68.39%  K-mer dist pass 1
00:00:00      2 MB(0%)  Iter   1   85.48%  K-mer dist pass 1
00:00:00      2 MB(0%)  Iter   1  100.00%  K-mer dist pass 1
00:00:00      2 MB(0%)  Iter   1    0.03%  K-mer dist pass 2
00:00:00      2 MB(0%)  Iter   1   17.12%  K-mer dist pass 2
00:00:00      2 MB(0%)  Iter   1   34.21%  K-mer dist pass 2
00:00:0

'results/1_orthofinder_analysis/HOG0002500_astR.muscle.aln'

In [50]:
# HOG0002500_protein_seqs = HOG0002500_protein_seqs.reset_index().set_index('Accession')
# HOG0002500_protein_seqs_figure = HOG0002500_protein_seqs.loc[figure_genome_assemblies.keys()].reset_index().set_index('index')

In [143]:
# aln_d = pfa.get_seq_dict(muscle_out)
# fig_aln_d = {k:v for k,v in aln_d.items() if k in HOG0002500_protein_seqs_figure.index}
fig_alignment_fp = results_dir / 'HOG0002500_astR_fig_accessions.aln'
# pfa.write_to_fasta(fig_aln_d, fig_alignment_fp)
aln_d = pfa.get_seq_dict(str(fig_alignment_fp))

### Load alignment and Run RAxML

In [62]:
raxml_out_dir = 'results/1_orthofinder_analysis/HOG0002500_astR_fig_accessions_raxML'
importlib.reload(ef)
ef.run_raxml(fig_alignment_fp, raxml_out_dir, prefix='AstR', threads=8, model='LG', raxml_bin='raxmlHPC')

Running: raxmlHPC -T 8 -m PROTGAMMALG -p 12345 -x 12345 -# 100 -f a -s /Users/jonathanwinkelman/Trestle/Palmer_lab/Geary_et_al_2025/results/1_orthofinder_analysis/HOG0002500_astR_fig_accessions.aln -n AstR -w /Users/jonathanwinkelman/Trestle/Palmer_lab/Geary_et_al_2025/results/1_orthofinder_analysis/HOG0002500_astR_fig_accessions_raxML
Option -T does not have any effect with the sequential or parallel MPI version.
It is used to specify the number of threads for the Pthreads-based parallelization
Warning, you specified a working directory via "-w"
Keep in mind that RAxML only accepts absolute path names, not relative ones!

RAxML can't, parse the alignment file as phylip file 
it will now try to parse it as FASTA file

IMPORTANT WARNING: Alignment column 5 contains only undetermined values which will be treated as missing data
IMPORTANT WARNING: Alignment column 6 contains only undetermined values which will be treated as missing data
IMPORTANT WARNING: Alignment column 7 contains only 

In [144]:
itol_annotations_dir = results_dir / 'itol_annotations'
itol_annotations_dir.mkdir(exist_ok=True)
tree = Phylo.read(file=f'{raxml_out_dir}/RAxML_bipartitionsBranchLabels.AstR', format='newick')
relable_d = {}
for cl in tree.get_terminals():
    acc = cl.name[:15][::-1].replace('_', '.', 1)[::-1]
    relable_d[cl.name] = dop_obj.accession_to_name[acc]
itol_relable_out = itol_annotations_dir / 'RELABLE_RAxML_bipartitionsBranchLabels.astR'
ia.relabel_itol_treeleafs(tree,relable_d, itol_relable_out )

In [116]:
### checking tree with larger number of AstR orthologs and running with IQtree

In [84]:
importlib.reload(ef)
iqtree_output_dir = results_dir / 'IQtree_astR'
iqtree_output_dir.mkdir(exist_ok=True)
ef.run_iqtree_asr(muscle_out,prefix='IQtree_astR', output_dir=iqtree_output_dir)

IQ-TREE multicore version 2.3.6 for MacOS ARM 64-bit built Aug  4 2024
Developed by Bui Quang Minh, Nguyen Lam Tung, Olga Chernomor, Heiko Schmidt,
Dominik Schrempf, Michael Woodhams, Ly Trong Nhan, Thomas Wong

Host:    mac.lan (SSE4.2, 36 GB RAM)
Command: iqtree2 -s results/1_orthofinder_analysis/HOG0002500_astR.muscle.aln -m MFP --ancestral -T 2 --alrt 1000 --prefix results/1_orthofinder_analysis/IQtree_astR/IQtree_astR
Seed:    812882 (Using SPRNG - Scalable Parallel Random Number Generator)
Time:    Fri Sep  5 11:31:40 2025
Kernel:  SSE2 - 2 threads (14 CPU cores detected)

Reading alignment file results/1_orthofinder_analysis/HOG0002500_astR.muscle.aln ... Fasta format detected
Reading fasta file: done in 0.000313997 secs using 80.26% CPU
Alignment most likely contains protein sequences
Alignment has 76 sequences with 171 columns, 169 distinct patterns
132 parsimony-informative, 17 singleton sites, 22 constant sites
                                Gap/Ambiguity  Composition  p-va

In [87]:
tree = Phylo.read(file=f'{iqtree_output_dir}/IQtree_astR.treefile', format='newick')
relable_d = {}
for cl in tree.get_terminals():
    acc = cl.name[:15][::-1].replace('_', '.', 1)[::-1]
    relable_d[cl.name] = dop_obj.accession_to_name[acc]
itol_relable_out = itol_annotations_dir / 'IQtree_astR.treefile'
ia.relabel_itol_treeleafs(tree,relable_d, itol_relable_out )